In this notebook, we train character-based LSTM models to generate Shakespearean-style sonnets.

In [1]:
import numpy as np

from DataProcessing import *
from RNNProcessing import *

import keras
from keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint

Using TensorFlow backend.


## Without punctuation and newlines
Read in Shakespearean sonnets with all punctuation and newlines removed, thus minimizing the number of input classes.

In [2]:
# Load data
sonnets, syllableDic = loadShake_char(stripPunc=True)
X_code, y_code, char2code, code2char = text2seq(sonnets)
X, y = seq2cat(X_code, y_code, len(char2code))

# Construct model
model = build_model_LSTM(X.shape[1:], y.shape[1])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               80896     
_________________________________________________________________
dense_1 (Dense)              (None, 29)                3741      
Total params: 84,637
Trainable params: 84,637
Non-trainable params: 0
_________________________________________________________________


Define callback functions for use in training.

In [3]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50)

def epoch_chars(epoch, _):
    # Generate a sequence of characters after each epoch
    print('--------------- Epoch %d Completed ---------------' % epoch)
    seed = X_code[np.random.choice(len(X_code))]
    T = 1
    seq_out = gen_chars(model, seed, code2char, n_chars=100, T=T, verbose=False)
    print(seq_out + '\n')

def epoch_lines(epoch, _):
    # Generate a sequence of lines after each epoch
    print('--------------- Epoch %d Completed ---------------' % epoch)
    seed = X_code[np.random.choice(len(X_code))]
    T = 1
    seq_out = gen_lines(model, seed, code2char, n_lines=2, T=T, verbose=False)
    print(seq_out + '\n')

chars_callback = LambdaCallback(on_epoch_end=epoch_chars)
lines_callback = LambdaCallback(on_epoch_end=epoch_lines)

Train model.

In [4]:
# Compile and fit model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, batch_size=64, epochs=20,
          validation_split=0.2, callbacks=[chars_callback],
          verbose=1)

# Save model
model.save('RNN_char-LSTM_no-punc.h5')

Instructions for updating:
Use tf.cast instead.
Train on 67713 samples, validate on 16929 samples
Epoch 1/20
67713/67713 [==============================] - 95s 1ms/step - loss: 2.4118 - acc: 0.3078 - val_loss: 2.1305 - val_acc: 0.3743
--------------- Epoch 0 Completed ---------------
hl wire for deaut o tloun t beorgangh spil prell tort she ios io hing ir of bing sor arougth be and 

Epoch 2/20
67713/67713 [==============================] - 89s 1ms/step - loss: 2.0225 - acc: 0.4012 - val_loss: 1.9599 - val_acc: 0.4156
--------------- Epoch 1 Completed ---------------
t the weattcowr canker coich aif the me weates not ikeds havess gurd indy brapk and and wey wall ans

Epoch 3/20
67713/67713 [==============================] - 89s 1ms/step - loss: 1.8896 - acc: 0.4361 - val_loss: 1.9003 - val_acc: 0.4270
--------------- Epoch 2 Completed ---------------
 deith sumerst mided sum'ss shape abe eters sing my yie that hos tany ses yous mall cun to se blose 

Epoch 4/20
67713/67713 [===========

Generate example text from trained model.

In [5]:
# Load model
model = keras.models.load_model('RNN_char-LSTM_no-punc.h5')

# Choose seed
seed = X_code[np.random.choice(len(X_code))]
print('Seed Sequence:')
print(''.join([code2char[code] for code in seed]))
print('')

# Generate sequences for different temperatures
for T in [0.5, 1, 1.5]:
    seq_out = gen_chars(model, seed, code2char, n_chars=1000, T=T,
                        verbose=False)
    print('Output Sequence for Temperature %.1f:' % T)
    print(seq_out)
    print('')

Seed Sequence:
 thy will and will to boot and will in o

Output Sequence for Temperature 0.5:
f thy self and thy past and thou be doth days and beauty's since the seem the wanter that my wide what i sum condect of hould in these can see since a change mine on thy self i joy thy hears breathe to be then to be then that they mayst like and the same and the serves the fince that is thy self the self do blessing mine own my grace which show thine own seeming the beauty's will i thine eye as the living beauty's painted what waster which lead the love that this thoughts on thee my love you be despite of shame were but the lease that thou decay so thou the still with the bear my love that thou art bearseness every with when i am and all your sall the time's thee their hadding thine of thee thy good than hampering thy self to break i for their brink nothing of the with they dead that though they then the world and the basest and this will to steal hear the world or all were borness and like an

## With punctuation and newlines
Read in Shakespearean sonnets with all punctuation and newlines preserved, enabling learning of the natural poem structure.

In [6]:
# Load data
sonnets, syllableDic = loadShake_char(stripPunc=False)
X_code, y_code, char2code, code2char = text2seq(sonnets)
X, y = seq2cat(X_code, y_code, len(char2code))

# Construct model
model = build_model_LSTM(X.shape[1:], y.shape[1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_2 (Dense)              (None, 38)                4902      
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


Train model, this time for up to 200 epochs.

In [7]:
# Add model checkpoint
model_checkpoint = ModelCheckpoint('RNN_char-LSTM.h5',
                                   monitor='val_loss', save_best_only=True)

# Compile and fit model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(X, y, batch_size=64, epochs=200,
                validation_split=0.2,
                callbacks=[early_stopping, model_checkpoint, lines_callback],
                verbose=1)

Train on 70503 samples, validate on 17626 samples
Epoch 1/200
70503/70503 [==============================] - 95s 1ms/step - loss: 2.5529 - acc: 0.2752 - val_loss: 2.2326 - val_acc: 0.3505
--------------- Epoch 0 Completed ---------------
sst arko iy inatt be qirltuncs ord wdtexk fes rand sith clan thith llleish dons raske dos id theugl co demes en pepllsd,
n wh mo mowemenfegron seofg it lo thakr

Epoch 2/200
70503/70503 [==============================] - 93s 1ms/step - loss: 2.1310 - acc: 0.3762 - val_loss: 2.0573 - val_acc: 0.3923
--------------- Epoch 1 Completed ---------------
odangs,
ind divej, on the prate

Epoch 3/200
70503/70503 [==============================] - 95s 1ms/step - loss: 1.9756 - acc: 0.4164 - val_loss: 1.9503 - val_acc: 0.4205
--------------- Epoch 2 Completed ---------------
 thy lizh's sonn,
ao suma, thet thou wastaye,

Epoch 4/200
70503/70503 [==============================] - 96s 1ms/step - loss: 1.8634 - acc: 0.4462 - val_loss: 1.8712 - val_acc: 0.4359
------

 nor may the speet be mone,
shall shorl is but thou art, wherein not so singen:

Epoch 33/200
70503/70503 [==============================] - 103s 1ms/step - loss: 1.0893 - acc: 0.6591 - val_loss: 1.9363 - val_acc: 0.4611
--------------- Epoch 32 Completed ---------------
 on, that moss wattire asb
then of thy report that weatare be not faie,

Epoch 34/200
70503/70503 [==============================] - 103s 1ms/step - loss: 1.0713 - acc: 0.6659 - val_loss: 1.9651 - val_acc: 0.4544
--------------- Epoch 33 Completed ---------------
 conot doth parch as field mure in sobles,
or dive and avingle breathings have doth prows,

Epoch 35/200
70503/70503 [==============================] - 103s 1ms/step - loss: 1.0557 - acc: 0.6705 - val_loss: 1.9800 - val_acc: 0.4548
--------------- Epoch 34 Completed ---------------
ratter sum.
cay her bicked eyes having tother in thy day,

Epoch 36/200
70503/70503 [==============================] - 103s 1ms/step - loss: 1.0401 - acc: 0.6758 - val_loss: 2.0031 

Generate example text from trained model.

In [8]:
# Load model
model = keras.models.load_model('RNN_char-LSTM.h5')

# Choose seed
seed = X_code[np.random.choice(len(X_code))]
print('Seed Sequence:')
print(''.join([code2char[code] for code in seed]))
print('')

# Generate sequences for different temperatures
for T in [0.5, 1, 1.5]:
    seq_out = gen_lines(model, seed, code2char, n_lines=14, T=T,
                        verbose=False)
    print('Output Sequence for Temperature %.1f:' % T)
    print(seq_out)
    print('')

Seed Sequence:
sed in giving gentle doom:
and taught it

Output Sequence for Temperature 0.5:
 see gove i prove your dessed,
with thou art so doth atate, and in the stare,
  then thou best but the state i hard showers thing and on still doth still,
the beauty that to the dear doth sweet speak,
when i for wist mich at the sparing thise in me.
  ending of self more that thou with thee,
and i all be the ofr and in praise,
and and in the time the sprain beauty,
and his the canss not the sore the have hor strong.
  but thou some doth say my sall the come,
and that i so should be the that for thee is doth friend.
  but this with thou art for my self my live,
  the come which not so sweet more of their infore,
  the store of make my strongen and of me,

Output Sequence for Temperature 1.0:
 not in my look, dear,
they nor heave grainejuct your live, and seech prase,
agioss to mame with not that your shame
by i desthing oo the faire, and the be:
  gave and and chatist your fair a prepe.
  i ape